In [73]:
import py2cytoscape
import networkx as nx
import pandas

In [6]:
from py2cytoscape.data.cyrest_client import CyRestClient

In [96]:
def mknode(name, level):
    return name, level
nodes = [
    mknode("Product", "L0"), 
    mknode("Market", "L0"),
    mknode("Regulatory", "L0"),
    mknode("Business", "L0"),
    mknode("System", "L0.5"),
    mknode("User", "L0.5"),
    mknode("Functional", "L1"),
    mknode("Performance", "L1"),
    mknode("Design", "L2"),
    mknode("Implementation", "L2"),
]

def add_edge(n1, n2, label):
    G.add_edge(n1, n2, link=label)

    
G = nx.DiGraph()
for node, level in nodes:
    G.add_node(node, level=level)

    
G = nx.DiGraph()

add_edge("L0", "L0", "Related Requirements")
add_edge("L1", "L1", "Related Requirements")
add_edge("L2", "L2", "Related Requirements")

add_edge("L0", "L0", "Requirement Decomposition")
add_edge("L1", "L1", "Functional Decomposition")
add_edge("L2", "L2", "Design Decomposition")

add_edge("L0", "L1", "Design Input Translation")
add_edge("L1", "L2", "Requirement Implementation")
add_edge("L0", "L2", "Requirement Implementation")



for l in "L0", "L1", "L2":
    add_edge("Bug", l, "Bug Fix")
    add_edge("Complaint", l, "External Request")
    add_edge("Feature Request", l, "External Request")
    add_edge("Audit Observation", l, "External Request")

In [33]:
cy = CyRestClient()
cy.session.delete()
data = py2cytoscape.util.from_networkx(G)

nw = cy.network.create(None, name="test", collection="Test1", data=data)

In [38]:
nw.get_node_table()

,SUID,id,name,selected,shared name
0,552,L1,L1,False,L1
1,553,L0,L0,False,L0
2,554,Complaint,Complaint,False,Complaint
3,555,L2,L2,False,L2
4,556,Bug,Bug,False,Bug
5,557,Audit Observation,Audit Observation,False,Audit Observation
6,558,Feature Request,Feature Request,False,Feature Request


In [97]:
nodes = ["L0", "L1", "L2", "Complaint", "Feature Request", "Audit Observation", "Bug"]
nnodes = len(nodes)
assert set(nodes) == set(G.nodes())

In [98]:
n2i = {n:i for i, n in enumerate(nodes)}

links = [["" for _ in range(nnodes)] for _ in range(nnodes)]
for e1, e2 in G.edges():
    link = G[e1][e2]['link']
    i1 = n2i[e1]
    i2 = n2i[e2]
    links[i2][i1] = link

In [99]:
df = pandas.DataFrame(links[:-4], index=nodes[:-4], columns=nodes)
df.dropna()
df

,L0,L1,L2,Complaint,Feature Request,Audit Observation,Bug
L0,Requirement Decomposition,,,External Request,External Request,External Request,Bug Fix
L1,Design Input Translation,Functional Decomposition,,External Request,External Request,External Request,Bug Fix
L2,Requirement Implementation,Requirement Implementation,Design Decomposition,External Request,External Request,External Request,Bug Fix


In [124]:
G = nx.DiGraph()
L0 = ["Product", "Market","Regulatory","Business", "System", "User"]
L1 = ["Functional", "Performance"]
L2 = ["Design", "Implementation", "Hardware", "Software"]

for l in L0, L1, L2:
    for ll in l:
        G.add_edge(ll, ll, link="Decomposition")
    
for n1 in L0:
    for n2 in L1:
        G.add_edge(n1, n2, link="Design Input Translation")

    for n2 in L2:
        G.add_edge(n1, n2, link="Requirement Implementation")
  
for n1 in L1:
    for n2 in L2:
        G.add_edge(n1, n2, link="Requirement Implementation")
 
import itertools
for n1 in ("Complaint", "Feature Request", "Audit Observation"):
    for n2 in itertools.chain(L0, L1, L2):
        G.add_edge(n1, n2, link="External Request")
        
for n2 in itertools.chain(L0, L1, L2):
    G.add_edge("Bug", n2, link="Bug Fix")
    
nodes = list(itertools.chain(L0, L1, L2, ("Complaint", "Feature Request", "Audit Observation", "Bug")))
nnodes = len(nodes)

n2i = {n:i for i, n in enumerate(nodes)}

links = [["" for _ in range(nnodes)] for _ in range(nnodes)]
for e1, e2 in G.edges():
    link = G[e1][e2]['link']
    i1 = n2i[e1]
    i2 = n2i[e2]
    links[i2][i1] = link

In [126]:
from officelib.xllib import *
xl = Excel()
wb =xl.Workbooks.Add()
cells = wb.Worksheets(1).Cells
cr = cells.Range
c1=cr("B2")
c2 = c1.GetOffset(len(links)-1, len(links[0])-1)
cr(c1,c2).Value2 = links

c3 = c1.GetOffset(-1, 0)
c4 = c3.GetOffset(0, len(links[0])-1)
cr(c3, c4).Value2 = nodes

c5 = c1.GetOffset(0, -1)
c6 = c1.GetOffset(len(links)-1, 0)

cr(c5, c6).Value2 = [(n,) for n in nodes]


In [116]:
df = pandas.DataFrame(links, index=nodes, columns=nodes)
df

,Product,Market,Regulatory,Business,System,User,Functional,Performance,Design,Implementation,Hardware,Software,Complaint,Feature Request,Audit Observation,Bug
Product,Decomposition,,,,,,,,,,,,External Request,External Request,External Request,Bug Fix
Market,,Decomposition,,,,,,,,,,,External Request,External Request,External Request,Bug Fix
Regulatory,,,Decomposition,,,,,,,,,,External Request,External Request,External Request,Bug Fix
Business,,,,Decomposition,,,,,,,,,External Request,External Request,External Request,Bug Fix
System,,,,,Decomposition,,,,,,,,External Request,External Request,External Request,Bug Fix
User,,,,,,Decomposition,,,,,,,External Request,External Request,External Request,Bug Fix
Functional,Design Input Translation,Design Input Translation,Design Input Translation,Design Input Translation,Design Input Translation,Design Input Translation,Decomposition,,,,,,External Request,External Request,External Request,Bug Fix
Performance,Design Input Translation,Design Input Translation,Design Input Translation,Design Input Translation,Design Input Translation,Design Input Translation,,Decomposition,,,,,External Request,External Request,External Request,Bug Fix
Design,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Decomposition,,,,External Request,External Request,External Request,Bug Fix
Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,Requirement Implementation,,Decomposition,,,External Request,External Request,External Request,Bug Fix


In [140]:
#cy = CyRestClient()
G = nx.DiGraph()
for i in range(1, 6):
    G.add_node(i, text="text:%d"%i)
G.add_edge(1, 2)
G.add_edge(2, 3)
G.add_edge(2, 4)
G.add_edge(4, 5)
# data = py2cytoscape.util.from_networkx(G)
# for node in data['elements']['nodes']:
#     if node['data']['name'] in ('1', '2', '3'):
#         node['data']['y'] = 0
#     elif node['data']['name'] == '4':
#         node['data']['y'] = -10
#     else:
#         node['data']['y'] = -20
        
# cy.session.delete()
# nw = cy.network.create(None, name="test", collection="Test1", data=data)

In [145]:
import io
fp = io.BytesIO()
nx.write_graphml_lxml(G, fp)
fp.getvalue()
from lxml import etree as lxml_etree
lxml_etree.dump(lxml_etree.XML(fp.getvalue()))
with open("text.graphml", 'wb') as f:
    f.write(fp.getvalue())

<graphml xmlns="http://graphml.graphdrawing.org/xmlns" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://graphml.graphdrawing.org/xmlns http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd"><key attr.name="text" attr.type="string" for="node" id="d0"/>
<graph edgedefault="directed"><node id="1">
  <data key="d0">text:1</data>
</node>
<node id="2">
  <data key="d0">text:2</data>
</node>
<node id="3">
  <data key="d0">text:3</data>
</node>
<node id="4">
  <data key="d0">text:4</data>
</node>
<node id="5">
  <data key="d0">text:5</data>
</node>
<edge source="1" target="2"/>
<edge source="2" target="3"/>
<edge source="2" target="4"/>
<edge source="4" target="5"/>
</graph></graphml>


In [201]:
fp = "pbs gamp5 #2.graphml"
tree = lxml_etree.parse(fp)
G = nx.read_graphml(fp)

root = tree.getroot()

textmap = {}
idmap = {}
nodemap = {}
for node in root.findall("graph/node", namespaces=root.nsmap):
    shapenode = node.find("data/y:ShapeNode", namespaces=root.nsmap)
    nodelabel = shapenode.find("y:NodeLabel", namespaces=root.nsmap)
    id = node.attrib['id']
    text = nodelabel.text.replace("\n", " ")
    textmap[text] = id
    idmap[id] = text
    nodemap[id] = shapenode
    
node1 = "Design, User, and Business Inputs"
id = textmap[node1]
sn = nodemap[id]

def geometry(n):
    return nodemap[n].find("y:Geometry", namespaces=root.nsmap).attrib

attrib = sn.find("y:Geometry", namespaces=root.nsmap).attrib
attrib["x"] = 0
#attrib["y"] = 0
seen = {id}
x = 0
for e1, e2 in G.edges():
    g1 = geometry(e1)
    g2 = geometry(e2)
    if e1 in seen:
        g2['x'] = g1[x] + 30
    elif e2 in seen:
        g1['x'] = g2['x'] - 30
    else:
    seen.add(e1, e2)

n0
n1
n2
n3
n4
n5
n6
n7


In [200]:
textmap.keys()

dict_keys(['Design, Interfaces,\nand Architecture', 'Implementation', 'Unit Testing', 'User Acceptance\nTesting', 'Integration Testing', 'Design, User,\nand Business Inputs', 'System Behavior and \nFeatures', 'System Testing'])

In [194]:
nodes=root.findall("graph/node", namespaces=root.nsmap)
nodes[0].findall("ShapeNode", namespaces=root.nsmap)

[]

In [198]:
list(nodes[0][1])

[<Element {http://www.yworks.com/xml/graphml}ShapeNode at 0x1ef33ba0148>]

In [182]:
for node in root.findall("graph/node/data/y:ShapeNode", namespaces=root.nsmap):
    print(repr(node.find("y:NodeLabel", namespaces=root.nsmap).text))

'Design, User,\nand Business Inputs'
'System Behavior and \nFeatures'
'Design, Interfaces,\nand Architecture'
'Implementation'
'User Acceptance\nTesting'
'System Testing'
'Integration Testing'
'Unit Testing'


In [176]:
G = nx.read_graphml("pbs gamp5 #2.graphml")

In [179]:
G.nodes()

NodeView(('n2', 'n1', 'n4', 'n0', 'n7', 'n5', 'n3', 'n6'))

In [274]:
nsmap = {
    'sys': 'http://www.yworks.com/xml/yfiles-common/markup/primitives/2.0',
    'y': 'http://www.yworks.com/xml/graphml',
    None: 'http://graphml.graphdrawing.org/xmlns',
    'x': 'http://www.yworks.com/xml/yfiles-common/markup/2.0',
    'yed': 'http://www.yworks.com/xml/yed/3',
    'java': 'http://www.yworks.com/xml/yfiles-common/1.0/java'
}

loc = "{http://www.w3.org/2001/XMLSchema-instance}schemaLocation"
schema = "http://www.yworks.com/xml/schema/graphml/1.1/ygraphml.xsd"

def key_attrname(aname, atyp, sfor, id):
    d = {
        'attr.name': aname,
        'attr.type': atyp,
        'for': sfor,
        'id': id
    }
    return lxml_etree.Element("key", d, nsmap)

def forkey(sfor, ytyp, id):
    d = {
        "for": sfor,
        "id": id,
        "yfiles.type": ytyp
    }
    return lxml_etree.Element("key", d, nsmap)

def data(kw=None, **kw2):
    if kw is not None:
        kw2.update(kw)
    return E("data", attrib=kw2, nsmap=nsmap)

import collections
__elem = collections.defaultdict(lambda: 0)
def uid_for(typ):
    uid = typ + str(__elem[typ])
    __elem[typ] += 1
    return uid
    
    
E = lxml_etree.Element
SE = lxml_etree.SubElement

root = lxml_etree.Element("graphml", attrib={loc: schema}, nsmap=nsmap)
root.append(key_attrname("Description", "string", "graph", "d0"))
root.append(key_attrname("url", "string", "node", "d1"))
root.append(key_attrname("description", "string", "node", "d2"))
root.append(key_attrname("url", "string", "edge", "d3"))
root.append(key_attrname("description", "string", "edge", "d4"))

root.append(forkey("node", "nodegraphics", "d5"))
root.append(forkey("graphml", "resources", "d6"))
root.append(forkey("edge", "edgegraphics", "d7"))

graph = E("graph", edgedefault="directed", id="G", nsmap=nsmap)
root.append(graph)

graph.append(data(key="d0"))

def make_node(graph, nid, text, x, y, color="#ffcc00"):
    node = SE(graph, "node", nsmap=nsmap, id=nid)
    sn = SE(node, "y:ShapeNode", nsmap=nsmap)
    SE(sn, "y:Geometry", nsmap=nsmap, height="40.0", width="128.0", x=x, y=y)
    SE(sn, "y:Fill", nsmap=nsmap, color=color, transparent=False)
    SE(sn, "y:BorderStyle", nsmap=nsmap, color="#000000", raised='false', type='line', width='1.0')
    SE(sn, "y:Shape", nsmap=nsmap, type='rectangle')
    nl_data = {
        'alignment': 'center',
        'autoSizePolicy': 'content',
        'fontFamily': 'Dialog',
        'fontStyle': "plain",
        'hasBackgroundColor': "false",
        "hasLineColor": "false",
        "horizontalTextPosition": "center",
        "textColor": "#000000",
        "verticalTextPosition": "bottom",
        "visible": "true",
        "xml:space": "preserve"
    }
    nl = SE(sn, "y:NodeLabel", nsmap=nsmap, attrib=nl_data)
    nl.text = text
    
def make_edge(graph, source, target):
    eid = uid_for("e")
    edge = SE(graph, "edge", id=eid, source=source, target=target, nsmap=nsmap)
    SE(edge, "data", nsmap=nsmap, key="d4")
    data = SE(edge, "data", nsmap=nsmap, key="d7")
    pl = SE(data, "y:PolyLineEdge", nsmap=nsmap)
    SE(pl, "y:Path", sx="0.0", sy="0.0", tx="0.0", ty="0.0", nsmap=nsmap)
    SE(pl, "y:LineStyle", color="#000000", type="line", width="1.0", nsmap=nsmap)
    SE(pl, "y:Arrows", source="standard", target="none", nsmap=nsmap)
    SE(pl, "y:BendStyle", smoothed="false")
    return eid



In [275]:
G = nx.DiGraph()

rtypes = ["Design, User,\nand Business Inputs", "System Behavior and\nFeatures", 
          "Design, Interfaces,\nand Architecture", "Implementation"]

ttypes = ["User Acceptance\nTesting", "System Testing", "Integration Testing", "Unit Testing"][::-1]

__nodes = {}
__nlabels = {}
def node_id(lbl):
    if lbl in __nodes:
        return __nodes[lbl]
    uid = uid_for('n')
    __nodes[lbl] = uid
    __nlabels[uid] = lbl
    return uid

def get_lbl(uid):
    return __nlabels[uid]

for a,b in zip(rtypes[:-1], rtypes[1:]):
    G.add_edge(node_id(a),node_id(b))
    
for a,b in zip(ttypes[:-1], rtypes[1:]):
    G.add_edge(node_id(a),node_id(b))
    
G.add_edge(rtypes[-1], ttypes[0])

for i, node in enumerate(rtypes):
    make_node(graph, node_id(node), i*30, i*60, "#9999FF")
    
for n, node in enumerate(ttypes):
    make_node(graph, node_id(node), (n+i+1)*30+100, (i - n)*60)
    
for e1, e2 in G.edges():
    make_edge(graph, e1, e2)

ValueError: Invalid tag name 'y:ShapeNode'

In [273]:
lxml_etree.tostring(root)

b'<graphml xmlns:java="http://www.yworks.com/xml/yfiles-common/1.0/java" xmlns:sys="http://www.yworks.com/xml/yfiles-common/markup/primitives/2.0" xmlns:x="http://www.yworks.com/xml/yfiles-common/markup/2.0" xmlns:y="http://www.yworks.com/xml/graphml" xmlns:yed="http://www.yworks.com/xml/yed/3" xmlns="http://graphml.graphdrawing.org/xmlns" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.yworks.com/xml/schema/graphml/1.1/ygraphml.xsd"><key attr.name="Description" attr.type="string" for="graph" id="d0"/><key attr.name="url" attr.type="string" for="node" id="d1"/><key attr.name="description" attr.type="string" for="node" id="d2"/><key attr.name="url" attr.type="string" for="edge" id="d3"/><key attr.name="description" attr.type="string" for="edge" id="d4"/><key for="node" id="d5" yfiles.type="nodegraphics"/><key for="graphml" id="d6" yfiles.type="resources"/><key for="edge" id="d7" yfiles.type="edgegraphics"/><graph edgedefault="directed" id="G"><data k